In [ ]:
from google.colab import files
uploaded = files.upload()

In [35]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential 
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D 
from keras.utils.np_utils import to_categorical

In [36]:
 #Looading the dataset
data = pd.read_csv('Sentiment.csv')
data = data[['text','sentiment']] # Keeping only the neccessary columns
data['text'] = data['text'].apply(lambda x: x.lower()) #converting to lower case
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))) #only a-z,A-Z,0-9 would be remaining in the data, else special characters are removed
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [37]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [38]:
batch_size = 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 27s - loss: 0.8289 - accuracy: 0.6469
0.7595441123969485
0.6795544028282166


In [39]:
# To Save the model 
model.save('smodel.h5')
from keras.models import load_model 
savedmodel = load_model('smodel.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [40]:
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = savedmodel.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 0:
  print("Neutral")
elif sentiment < 0:
  print("Negative")
elif sentiment > 0:
  print("Positive")
else:
  print("Can not be determined")

0
Neutral


In [41]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20, 40]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train,Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 42s - loss: 0.8309 - accuracy: 0.6494
Epoch 1/1
 - 39s - loss: 0.8273 - accuracy: 0.6462
Epoch 1/1
 - 39s - loss: 0.8312 - accuracy: 0.6443
Epoch 1/1
 - 39s - loss: 0.8287 - accuracy: 0.6402
Epoch 1/1
 - 40s - loss: 0.8239 - accuracy: 0.6488
Epoch 1/2
 - 40s - loss: 0.8318 - accuracy: 0.6439
Epoch 2/2
 - 40s - loss: 0.6908 - accuracy: 0.7101
Epoch 1/2
 - 39s - loss: 0.8271 - accuracy: 0.6454
Epoch 2/2
 - 38s - loss: 0.6986 - accuracy: 0.7039
Epoch 1/2
 - 44s - loss: 0.8298 - accuracy: 0.6462
Epoch 2/2
 - 41s - loss: 0.6886 - accuracy: 0.7072
Epoch 1/2
 - 40s - loss: 0.8316 - accuracy: 0.6409
Epoch 2/2
 - 41s - loss: 0.6837 - accuracy: 0.7112
Epoch 1/2
 - 39s - loss: 0.8263 - accuracy: 0.6443
Epoch 2/2
 - 40s - loss: 0.6771 - accuracy: 0.7120
Epoch 1/1
 - 27s - loss: 0.8341 - accuracy: 0.6411
Epoch 1/1
 - 27s - loss: 0.8297 - accuracy: 0.6408
Epoch 1/1
 - 27s - loss: 0.8421 - accuracy: 0.6329
Epoch 1/1
 - 28s - loss: 0.8354 - accuracy: 0.6399
Epoch 1/1
 - 29s - loss: 0.8316